## start of chapter 2

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
print(torch.cuda.is_available())

True


In [3]:
weight=0.7
bias=0.3
start=0
end=1
step=0.02
x=torch.arange(start,end,step).unsqueeze(dim=1)
y=weight*x+bias
x[:10],y[:10],len(x),len(y),x.shape,y.shape,len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50,
 torch.Size([50, 1]),
 torch.Size([50, 1]),
 50)

the ratio between the traning set and the validation set is <br>
*traning set : 60%-80%<br>
*validation set : 10%-20%<br>
*test set : 10%-20%<br>